In [8]:
import numpy as np
from pathlib import Path
import os
import cv2
import pickle
from pycocotools.coco import COCO
import patch as pat
import shutil

import skimage.transform
import numpy as np
from PIL import Image

#https://blog.csdn.net/qq_38048756/article/details/103208834
domain = 'train/'

dataroot = 'FLIR_people_select/' 

jsonroot = 'FLIR_ADAS_1_3/' 
jsonfile = jsonroot + domain + 'thermal_annotations.json'
coco = COCO(jsonfile)

# path to store file
# path =

#domain = 'train/'


C:\Users\dell\Anaconda3\envs\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dell\Anaconda3\envs\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\dell\Anaconda3\envs\venv\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [9]:
!pip uninstall numpy
!pip install numpy

^C


In [7]:
!pip install tensorflow --upgrade

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] 拒绝访问。: 'C:\\Users\\dell\\AppData\\Roaming\\Python\\Python38\\site-packages\\~umpy\\.libs\\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached numpy-1.19.5-cp38-cp38-win_amd64.whl (13.3 MB)
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30693 sha256=43c5bb087384d18eb78377a7b07ef539d2ba40736eaaaebf33d7fecf35c46bde
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\f1\60\77\22b9b5887bd47801796a856f47650d9789c74dc3161a26d608
Successfully built clang
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.1
    Uninstalling numpy-1.21.1:
      Successfully uninstalled numpy-1.21.1
Processing c:\users\dell\appdata\local\pip\cache\wheels\f1\60\77\22b9b5887bd47801796a856f47650d9789c74dc3161a26d608\clang-5.0-py3-none-

ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\venv\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "C:\Users\dell\Anaconda3\envs\venv\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\dell\Anaconda3\envs\venv\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "C:\Users\dell\Anaconda3\envs\venv\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\dell\Ana


  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Can't uninstall 'six'. No files were found to uninstall.
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.1
    Can't uninstall 'grpcio'. No files were found to uninstall.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.1
    Can't uninstall 'numpy'. No files were found to uninstall.
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0



ERROR: Could not install packages due to an EnvironmentError: [WinError 5] 拒绝访问。: 'C:\\Users\\dell\\Anaconda3\\envs\\venv\\lib\\site-packages\\~ensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



In [14]:
!git clone https://github.com/ultralytics/yolov3  # clone repo
%cd yolov3
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cpu (CPU)


In [17]:
### generate patches for a single image 
def valid_ann(ann):
    if ann['category_id'] == 1 and ann['bbox'][3]>120:
        return True
    else:
        return False

def generate_patch(idn,path_in_str,s,sig,angle,centers):
    imInfo = coco.imgs[idn]
    annIds = coco.getAnnIds(imgIds=imInfo['id']) #get annotations id of this image
    anns = coco.loadAnns(annIds)
    img = cv2.imread(path_in_str,0)
    for person_ann in anns:
        print(person_ann )
        if valid_ann(person_ann) == True:
            x,y,w,h = pat.patch_range(person_ann['bbox'],sig)
            print(x,y,w,h)
            #delete the rectangle when necessary
            cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
            box_size = (w,h)
            patch = pat.multiple_gussians(centers,s,sig ,box_size, x ,y)
            print(patch.shape)
            
            name = path_in_str[-15:-5]
            save_path = 'patched_image/train/'+ name
            
            #patche one
            im_p = pat.patched_image(img,patch,x,y,w,h)
            im = Image.fromarray(np.uint8(im_p))
            im.save(save_path + '.jpeg','JPEG')
            
            #rotated patch
            patch_rotate = pat.rand_rotate(patch,angle)
            im_p = pat.patched_image(img, patch_rotate,x,y,w,h)
            im = Image.fromarray(np.uint8(im_p))
            im.save(save_path + '_r.jpeg','JPEG')
            
            #bright-changed patch
            patch_bright = pat.adjust_contrast_bri(contrast,brightness,patch)
            im_p = pat.patched_image(img, patch_bright ,x,y,w,h)
            im = Image.fromarray(np.uint8(im_p))
            im.save(save_path + '_b.jpeg','JPEG')
        else:
            continue

        
#generate patches for image in a domain
def generate_patches(centers,domain,s,sig,rotate_ran):
    if domain == 'train/':
        with open ('FLIR_people_select_trainIdx', 'rb') as fp:
            idxs = pickle.load(fp)
    elif domain =='val/':
        with open ('FLIR_people_select_valIdx', 'rb') as fp:
            idxs = pickle.load(fp)
    #patch_sum = patch.multiple_gussians(centers)
    path = dataroot + domain 
    pathlist = Path(path).rglob('*.jpeg')
    for path,idn in zip(pathlist,idxs):
        path_in_str = str(path)
        print(path_in_str)
        generate_patch(idn,path_in_str,s,sig,rotate_ran,centers)
        #save image in FLIR/patched_image/train
    
    #file path of the patched images  
    return 

def yolo_scores(path):
    pathlist = Path(path).rglob('*.txt')
    scores_yolo = list()
    images = list()
    count = 0
    for path in pathlist:
        count += 1
        image = list()
        f = open(path,'r')
        lines = f.readlines()
        for line in lines:
            num_list = line.split()
            #print(num_list)
            if num_list[0] == "0":
                image.append(float(num_list[-1]))  
        if count == 4:
            scores_yolo.append(images)
            images = list()
            count = 0
        else:
            images.append(image)
    return scores_yolo
                
def yolov3_scores(centers):
    patch_images = generate_patches(centers) #path to folders store the images
    ...
    #use terminal command instead of function
    return scores # a list
def faster_rcnn_scores():
    generate_patches(centers)
    ...
    return scores # alist
def loss_function(centers):
    detector1_scores = yolov3_scores(centers)
    #detector2_scores = faster_rcnn_scores(centers)
    #L1_detector = 


In [7]:
class FocalLoss(nn.Module):
    # Wraps focal loss around existing loss_fcn(), i.e. criteria = FocalLoss(nn.BCEWithLogitsLoss(), gamma=1.5)
    def __init__(self, loss_fcn, gamma=1.5, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.loss_fcn = loss_fcn  # must be nn.BCEWithLogitsLoss()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = loss_fcn.reduction
        self.loss_fcn.reduction = 'none'  # required to apply FL to each element

    def forward(self, pred, true):
        loss = self.loss_fcn(pred, true)
        # p_t = torch.exp(-loss)
        # loss *= self.alpha * (1.000001 - p_t) ** self.gamma  # non-zero power for gradient stability

        # TF implementation https://github.com/tensorflow/addons/blob/v0.7.1/tensorflow_addons/losses/focal_loss.py
        pred_prob = torch.sigmoid(pred)  # prob from logits
        p_t = true * pred_prob + (1 - true) * (1 - pred_prob)
        alpha_factor = true * self.alpha + (1 - true) * (1 - self.alpha)
        modulating_factor = (1.0 - p_t) ** self.gamma
        loss *= alpha_factor * modulating_factor

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:  # 'none'
            return loss
BCEobj = nn.BCEWithLogitsLoss()
BCEobj = FocalLoss(BCEobj, gamma = 1.5)

In [30]:
import torch.nn as nn
import keras.backend as K
from torch import tensor 
centers = [0.05,0.3,0.35,0.6,0.67,0.9]
target = 0 #to minimize score, maximize loss
loss = K.categorical_crossentropy(tensor([0]), tensor([0.3]))
grads = K.gradients(loss,tensor(centers))

InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a int64 tensor but is a float tensor [Op:Mul]

In [25]:
# save patched image label for yolov3
if domain == 'train/':
    with open ('FLIR_people_select_trainIdx', 'rb') as fp:
        idxs = pickle.load(fp)
elif domain =='val/':
    with open ('FLIR_people_select_valIdx', 'rb') as fp:
        idxs = pickle.load(fp)
ran = idxs 
src_train = "FLIR_people_select/train/labels/"
dst_train = "patched_image/train/labels/"

for newidx, imageidx in enumerate(ran):
    img = coco.loadImgs(ids = [imageidx])
    file_name = img[0]['file_name']
    name = file_name[-15:-5] 
    print(name)
    src_dir = src_train + name + '.txt'
    dst_dir1 = dst_train + name + '.txt'
    shutil.copy(src_dir,dst_dir1)
    dst_dir2 = dst_train + name + '_b.txt'
    shutil.copy(src_dir,dst_dir2)
    dst_dir3 = dst_train + name + '_r.txt'
    shutil.copy(src_dir,dst_dir3)
    print(dst_dir1,dst_dir2,dst_dir3)

FLIR_00320
patched_image/train/labels/FLIR_00320.txt patched_image/train/labels/FLIR_00320_b.txt patched_image/train/labels/FLIR_00320_r.txt
FLIR_00377
patched_image/train/labels/FLIR_00377.txt patched_image/train/labels/FLIR_00377_b.txt patched_image/train/labels/FLIR_00377_r.txt
FLIR_00416
patched_image/train/labels/FLIR_00416.txt patched_image/train/labels/FLIR_00416_b.txt patched_image/train/labels/FLIR_00416_r.txt
FLIR_00695
patched_image/train/labels/FLIR_00695.txt patched_image/train/labels/FLIR_00695_b.txt patched_image/train/labels/FLIR_00695_r.txt
FLIR_00922
patched_image/train/labels/FLIR_00922.txt patched_image/train/labels/FLIR_00922_b.txt patched_image/train/labels/FLIR_00922_r.txt
FLIR_00927
patched_image/train/labels/FLIR_00927.txt patched_image/train/labels/FLIR_00927_b.txt patched_image/train/labels/FLIR_00927_r.txt
FLIR_00965
patched_image/train/labels/FLIR_00965.txt patched_image/train/labels/FLIR_00965_b.txt patched_image/train/labels/FLIR_00965_r.txt
FLIR_01500
pa

patched_image/train/labels/FLIR_01915.txt patched_image/train/labels/FLIR_01915_b.txt patched_image/train/labels/FLIR_01915_r.txt
FLIR_01916
patched_image/train/labels/FLIR_01916.txt patched_image/train/labels/FLIR_01916_b.txt patched_image/train/labels/FLIR_01916_r.txt
FLIR_01917
patched_image/train/labels/FLIR_01917.txt patched_image/train/labels/FLIR_01917_b.txt patched_image/train/labels/FLIR_01917_r.txt
FLIR_01918
patched_image/train/labels/FLIR_01918.txt patched_image/train/labels/FLIR_01918_b.txt patched_image/train/labels/FLIR_01918_r.txt
FLIR_01921
patched_image/train/labels/FLIR_01921.txt patched_image/train/labels/FLIR_01921_b.txt patched_image/train/labels/FLIR_01921_r.txt
FLIR_01922
patched_image/train/labels/FLIR_01922.txt patched_image/train/labels/FLIR_01922_b.txt patched_image/train/labels/FLIR_01922_r.txt
FLIR_01923
patched_image/train/labels/FLIR_01923.txt patched_image/train/labels/FLIR_01923_b.txt patched_image/train/labels/FLIR_01923_r.txt
FLIR_01924
patched_image

patched_image/train/labels/FLIR_02141.txt patched_image/train/labels/FLIR_02141_b.txt patched_image/train/labels/FLIR_02141_r.txt
FLIR_02142
patched_image/train/labels/FLIR_02142.txt patched_image/train/labels/FLIR_02142_b.txt patched_image/train/labels/FLIR_02142_r.txt
FLIR_02188
patched_image/train/labels/FLIR_02188.txt patched_image/train/labels/FLIR_02188_b.txt patched_image/train/labels/FLIR_02188_r.txt
FLIR_02211
patched_image/train/labels/FLIR_02211.txt patched_image/train/labels/FLIR_02211_b.txt patched_image/train/labels/FLIR_02211_r.txt
FLIR_02212
patched_image/train/labels/FLIR_02212.txt patched_image/train/labels/FLIR_02212_b.txt patched_image/train/labels/FLIR_02212_r.txt
FLIR_02213
patched_image/train/labels/FLIR_02213.txt patched_image/train/labels/FLIR_02213_b.txt patched_image/train/labels/FLIR_02213_r.txt
FLIR_02214
patched_image/train/labels/FLIR_02214.txt patched_image/train/labels/FLIR_02214_b.txt patched_image/train/labels/FLIR_02214_r.txt
FLIR_02215
patched_image

patched_image/train/labels/FLIR_02943.txt patched_image/train/labels/FLIR_02943_b.txt patched_image/train/labels/FLIR_02943_r.txt
FLIR_02950
patched_image/train/labels/FLIR_02950.txt patched_image/train/labels/FLIR_02950_b.txt patched_image/train/labels/FLIR_02950_r.txt
FLIR_02951
patched_image/train/labels/FLIR_02951.txt patched_image/train/labels/FLIR_02951_b.txt patched_image/train/labels/FLIR_02951_r.txt
FLIR_02952
patched_image/train/labels/FLIR_02952.txt patched_image/train/labels/FLIR_02952_b.txt patched_image/train/labels/FLIR_02952_r.txt
FLIR_02954
patched_image/train/labels/FLIR_02954.txt patched_image/train/labels/FLIR_02954_b.txt patched_image/train/labels/FLIR_02954_r.txt
FLIR_02955
patched_image/train/labels/FLIR_02955.txt patched_image/train/labels/FLIR_02955_b.txt patched_image/train/labels/FLIR_02955_r.txt
FLIR_02956
patched_image/train/labels/FLIR_02956.txt patched_image/train/labels/FLIR_02956_b.txt patched_image/train/labels/FLIR_02956_r.txt
FLIR_02957
patched_image

patched_image/train/labels/FLIR_03227.txt patched_image/train/labels/FLIR_03227_b.txt patched_image/train/labels/FLIR_03227_r.txt
FLIR_03228
patched_image/train/labels/FLIR_03228.txt patched_image/train/labels/FLIR_03228_b.txt patched_image/train/labels/FLIR_03228_r.txt
FLIR_03229
patched_image/train/labels/FLIR_03229.txt patched_image/train/labels/FLIR_03229_b.txt patched_image/train/labels/FLIR_03229_r.txt
FLIR_03230
patched_image/train/labels/FLIR_03230.txt patched_image/train/labels/FLIR_03230_b.txt patched_image/train/labels/FLIR_03230_r.txt
FLIR_03231
patched_image/train/labels/FLIR_03231.txt patched_image/train/labels/FLIR_03231_b.txt patched_image/train/labels/FLIR_03231_r.txt
FLIR_03232
patched_image/train/labels/FLIR_03232.txt patched_image/train/labels/FLIR_03232_b.txt patched_image/train/labels/FLIR_03232_r.txt
FLIR_03233
patched_image/train/labels/FLIR_03233.txt patched_image/train/labels/FLIR_03233_b.txt patched_image/train/labels/FLIR_03233_r.txt
FLIR_03234
patched_image

...

patched_image/train/labels/FLIR_06565.txt patched_image/train/labels/FLIR_06565_b.txt patched_image/train/labels/FLIR_06565_r.txt
FLIR_06566
patched_image/train/labels/FLIR_06566.txt patched_image/train/labels/FLIR_06566_b.txt patched_image/train/labels/FLIR_06566_r.txt
FLIR_06567
patched_image/train/labels/FLIR_06567.txt patched_image/train/labels/FLIR_06567_b.txt patched_image/train/labels/FLIR_06567_r.txt
FLIR_06568
patched_image/train/labels/FLIR_06568.txt patched_image/train/labels/FLIR_06568_b.txt patched_image/train/labels/FLIR_06568_r.txt
FLIR_06569
patched_image/train/labels/FLIR_06569.txt patched_image/train/labels/FLIR_06569_b.txt patched_image/train/labels/FLIR_06569_r.txt
FLIR_06570
patched_image/train/labels/FLIR_06570.txt patched_image/train/labels/FLIR_06570_b.txt patched_image/train/labels/FLIR_06570_r.txt
FLIR_06571
patched_image/train/labels/FLIR_06571.txt patched_image/train/labels/FLIR_06571_b.txt patched_image/train/labels/FLIR_06571_r.txt
FLIR_06572
patched_image

patched_image/train/labels/FLIR_06840.txt patched_image/train/labels/FLIR_06840_b.txt patched_image/train/labels/FLIR_06840_r.txt
FLIR_06841
patched_image/train/labels/FLIR_06841.txt patched_image/train/labels/FLIR_06841_b.txt patched_image/train/labels/FLIR_06841_r.txt
FLIR_06842
patched_image/train/labels/FLIR_06842.txt patched_image/train/labels/FLIR_06842_b.txt patched_image/train/labels/FLIR_06842_r.txt
FLIR_06858
patched_image/train/labels/FLIR_06858.txt patched_image/train/labels/FLIR_06858_b.txt patched_image/train/labels/FLIR_06858_r.txt
FLIR_06894
patched_image/train/labels/FLIR_06894.txt patched_image/train/labels/FLIR_06894_b.txt patched_image/train/labels/FLIR_06894_r.txt
FLIR_06895
patched_image/train/labels/FLIR_06895.txt patched_image/train/labels/FLIR_06895_b.txt patched_image/train/labels/FLIR_06895_r.txt
FLIR_06896
patched_image/train/labels/FLIR_06896.txt patched_image/train/labels/FLIR_06896_b.txt patched_image/train/labels/FLIR_06896_r.txt
FLIR_06897
patched_image

patched_image/train/labels/FLIR_07833.txt patched_image/train/labels/FLIR_07833_b.txt patched_image/train/labels/FLIR_07833_r.txt
FLIR_07834
patched_image/train/labels/FLIR_07834.txt patched_image/train/labels/FLIR_07834_b.txt patched_image/train/labels/FLIR_07834_r.txt
FLIR_07869
patched_image/train/labels/FLIR_07869.txt patched_image/train/labels/FLIR_07869_b.txt patched_image/train/labels/FLIR_07869_r.txt
FLIR_07878
patched_image/train/labels/FLIR_07878.txt patched_image/train/labels/FLIR_07878_b.txt patched_image/train/labels/FLIR_07878_r.txt
FLIR_07968
patched_image/train/labels/FLIR_07968.txt patched_image/train/labels/FLIR_07968_b.txt patched_image/train/labels/FLIR_07968_r.txt
FLIR_07969
patched_image/train/labels/FLIR_07969.txt patched_image/train/labels/FLIR_07969_b.txt patched_image/train/labels/FLIR_07969_r.txt
FLIR_07970
patched_image/train/labels/FLIR_07970.txt patched_image/train/labels/FLIR_07970_b.txt patched_image/train/labels/FLIR_07970_r.txt
FLIR_07971
patched_image

patched_image/train/labels/FLIR_08253.txt patched_image/train/labels/FLIR_08253_b.txt patched_image/train/labels/FLIR_08253_r.txt
FLIR_08254
patched_image/train/labels/FLIR_08254.txt patched_image/train/labels/FLIR_08254_b.txt patched_image/train/labels/FLIR_08254_r.txt
FLIR_08265
patched_image/train/labels/FLIR_08265.txt patched_image/train/labels/FLIR_08265_b.txt patched_image/train/labels/FLIR_08265_r.txt
FLIR_08266
patched_image/train/labels/FLIR_08266.txt patched_image/train/labels/FLIR_08266_b.txt patched_image/train/labels/FLIR_08266_r.txt
FLIR_08267
patched_image/train/labels/FLIR_08267.txt patched_image/train/labels/FLIR_08267_b.txt patched_image/train/labels/FLIR_08267_r.txt
FLIR_08268
patched_image/train/labels/FLIR_08268.txt patched_image/train/labels/FLIR_08268_b.txt patched_image/train/labels/FLIR_08268_r.txt
FLIR_08269
patched_image/train/labels/FLIR_08269.txt patched_image/train/labels/FLIR_08269_b.txt patched_image/train/labels/FLIR_08269_r.txt
FLIR_08270
patched_image

patched_image/train/labels/FLIR_08491.txt patched_image/train/labels/FLIR_08491_b.txt patched_image/train/labels/FLIR_08491_r.txt
FLIR_08492
patched_image/train/labels/FLIR_08492.txt patched_image/train/labels/FLIR_08492_b.txt patched_image/train/labels/FLIR_08492_r.txt
FLIR_08493
patched_image/train/labels/FLIR_08493.txt patched_image/train/labels/FLIR_08493_b.txt patched_image/train/labels/FLIR_08493_r.txt
FLIR_08494
patched_image/train/labels/FLIR_08494.txt patched_image/train/labels/FLIR_08494_b.txt patched_image/train/labels/FLIR_08494_r.txt
FLIR_08495
patched_image/train/labels/FLIR_08495.txt patched_image/train/labels/FLIR_08495_b.txt patched_image/train/labels/FLIR_08495_r.txt
FLIR_08496
patched_image/train/labels/FLIR_08496.txt patched_image/train/labels/FLIR_08496_b.txt patched_image/train/labels/FLIR_08496_r.txt
FLIR_08497
patched_image/train/labels/FLIR_08497.txt patched_image/train/labels/FLIR_08497_b.txt patched_image/train/labels/FLIR_08497_r.txt
FLIR_08540
patched_image

In [24]:
idxs

[319,
 376,
 415,
 694,
 921,
 926,
 964,
 1499,
 1510,
 1661,
 1662,
 1663,
 1664,
 1665,
 1666,
 1667,
 1668,
 1669,
 1670,
 1671,
 1672,
 1673,
 1674,
 1675,
 1676,
 1677,
 1678,
 1679,
 1680,
 1681,
 1682,
 1683,
 1684,
 1685,
 1686,
 1687,
 1688,
 1689,
 1691,
 1696,
 1697,
 1698,
 1703,
 1721,
 1722,
 1728,
 1730,
 1825,
 1831,
 1832,
 1848,
 1849,
 1857,
 1864,
 1867,
 1868,
 1896,
 1898,
 1899,
 1907,
 1914,
 1915,
 1916,
 1917,
 1920,
 1921,
 1922,
 1923,
 1924,
 1927,
 1940,
 1941,
 1942,
 1943,
 1958,
 1959,
 1981,
 1997,
 1998,
 1999,
 2002,
 2003,
 2004,
 2005,
 2010,
 2011,
 2019,
 2021,
 2059,
 2067,
 2073,
 2078,
 2079,
 2080,
 2081,
 2082,
 2088,
 2089,
 2090,
 2091,
 2092,
 2093,
 2094,
 2095,
 2096,
 2097,
 2098,
 2101,
 2102,
 2103,
 2108,
 2109,
 2129,
 2133,
 2134,
 2135,
 2136,
 2137,
 2138,
 2139,
 2140,
 2141,
 2187,
 2210,
 2211,
 2212,
 2213,
 2214,
 2215,
 2216,
 2242,
 2304,
 2307,
 2319,
 2354,
 2355,
 2395,
 2396,
 2398,
 2448,
 2449,
 2450,
 2485,
 2530,

In [18]:
names = list()
'''
def generate_patch(path):
    pathlist = Path(path).rglob('*.jpeg')
    for path in pathlist:
        path_in_str = str(path)
        print(path)
        names.append(path_in_str)

dataroot = '/home/hwjin/FLIR/yolov3/FLIR_people_select/' + os.sep
generate_patch(dataroot + 'val/')

'''

centers = [0.05,0.3,0.35,0.6,0.67,0.9]

s = 10
sig = 5.07
rotate_ran = 20

contrast = 30
brightness = 50

generate_patches(centers,domain,s,sig,rotate_ran)


FileNotFoundError: [Errno 2] No such file or directory: 'FLIR_people_select_trainIdx'

In [4]:
if domain == 'train/':
    with open ('FLIR_people_select_trainIdx', 'rb') as fp:
        idxs = pickle.load(fp)
        
test = list()
for id in idxs:
    imginfo = coco.imgs[id]
    test.append(imginfo['file_name'])

In [5]:
test

['thermal_8_bit/FLIR_00320.jpeg',
 'thermal_8_bit/FLIR_00377.jpeg',
 'thermal_8_bit/FLIR_00416.jpeg',
 'thermal_8_bit/FLIR_00695.jpeg',
 'thermal_8_bit/FLIR_00922.jpeg',
 'thermal_8_bit/FLIR_00927.jpeg',
 'thermal_8_bit/FLIR_00965.jpeg',
 'thermal_8_bit/FLIR_01500.jpeg',
 'thermal_8_bit/FLIR_01511.jpeg',
 'thermal_8_bit/FLIR_01662.jpeg',
 'thermal_8_bit/FLIR_01663.jpeg',
 'thermal_8_bit/FLIR_01664.jpeg',
 'thermal_8_bit/FLIR_01665.jpeg',
 'thermal_8_bit/FLIR_01666.jpeg',
 'thermal_8_bit/FLIR_01667.jpeg',
 'thermal_8_bit/FLIR_01668.jpeg',
 'thermal_8_bit/FLIR_01669.jpeg',
 'thermal_8_bit/FLIR_01670.jpeg',
 'thermal_8_bit/FLIR_01671.jpeg',
 'thermal_8_bit/FLIR_01672.jpeg',
 'thermal_8_bit/FLIR_01673.jpeg',
 'thermal_8_bit/FLIR_01674.jpeg',
 'thermal_8_bit/FLIR_01675.jpeg',
 'thermal_8_bit/FLIR_01676.jpeg',
 'thermal_8_bit/FLIR_01677.jpeg',
 'thermal_8_bit/FLIR_01678.jpeg',
 'thermal_8_bit/FLIR_01679.jpeg',
 'thermal_8_bit/FLIR_01680.jpeg',
 'thermal_8_bit/FLIR_01681.jpeg',
 'thermal_8_bi

In [6]:
path = dataroot + domain 
pathlist = Path(path).rglob('*.jpeg')
for path,idn in zip(pathlist,idxs):
    print(path)

FLIR_people_select\train\images\FLIR_00320.jpeg
FLIR_people_select\train\images\FLIR_00377.jpeg
FLIR_people_select\train\images\FLIR_00416.jpeg
FLIR_people_select\train\images\FLIR_00695.jpeg
FLIR_people_select\train\images\FLIR_00922.jpeg
FLIR_people_select\train\images\FLIR_00927.jpeg
FLIR_people_select\train\images\FLIR_00965.jpeg
FLIR_people_select\train\images\FLIR_01500.jpeg
FLIR_people_select\train\images\FLIR_01511.jpeg
FLIR_people_select\train\images\FLIR_01662.jpeg
FLIR_people_select\train\images\FLIR_01663.jpeg
FLIR_people_select\train\images\FLIR_01664.jpeg
FLIR_people_select\train\images\FLIR_01665.jpeg
FLIR_people_select\train\images\FLIR_01666.jpeg
FLIR_people_select\train\images\FLIR_01667.jpeg
FLIR_people_select\train\images\FLIR_01668.jpeg
FLIR_people_select\train\images\FLIR_01669.jpeg
FLIR_people_select\train\images\FLIR_01670.jpeg
FLIR_people_select\train\images\FLIR_01671.jpeg
FLIR_people_select\train\images\FLIR_01672.jpeg
FLIR_people_select\train\images\FLIR_016

FLIR_people_select\train\images\FLIR_08541.jpeg
FLIR_people_select\train\images\FLIR_08764.jpeg
FLIR_people_select\train\images\FLIR_08765.jpeg
FLIR_people_select\train\images\FLIR_08766.jpeg
FLIR_people_select\train\images\FLIR_08767.jpeg
FLIR_people_select\train\images\FLIR_08768.jpeg
FLIR_people_select\train\images\FLIR_08769.jpeg
FLIR_people_select\train\images\FLIR_08770.jpeg
FLIR_people_select\train\images\FLIR_08771.jpeg
FLIR_people_select\train\images\FLIR_08811.jpeg
FLIR_people_select\train\images\FLIR_08815.jpeg
FLIR_people_select\train\images\FLIR_08816.jpeg


In [10]:
#yolov3 detect inference patched image
!python detect.py --weights yolov3.pt --img 640 --conf 0.25 --save-conf --save-txt --source patched_image/train/images

#Image(filename='runs/detect/exp/zidane.jpg', width=600)

^C


In [25]:
path = 'runs/detect/exp11/labels'

In [17]:
    pathlist = Path(path).rglob('*.txt')
    scores_yolo = list()
    for path in pathlist:
        f = open(path,'r')
        lines = f.readlines()
        for line in lines:
            num_list = line.split()
            print(num_list)
            if num_list[0] == "0":
                scores_yolo.append(float(num_list[-1]))       

['5', '0.03125', '0.454102', '0.0625', '0.138672', '0.296153']
['9', '0.586719', '0.165039', '0.0296875', '0.0761719', '0.672735']
['2', '0.920313', '0.50293', '0.159375', '0.130859', '0.716543']
['2', '0.36875', '0.504883', '0.303125', '0.130859', '0.782924']
['0', '0.207813', '0.526367', '0.15', '0.330078', '0.862248']
['5', '0.03125', '0.454102', '0.0625', '0.134766', '0.326814']
['9', '0.586719', '0.165039', '0.0296875', '0.0761719', '0.672424']
['2', '0.920313', '0.50293', '0.159375', '0.130859', '0.716543']
['2', '0.367188', '0.504883', '0.30625', '0.130859', '0.759208']
['0', '0.207031', '0.527344', '0.151562', '0.328125', '0.847822']
['5', '0.03125', '0.454102', '0.0625', '0.138672', '0.296335']
['9', '0.586719', '0.165039', '0.0296875', '0.0761719', '0.672714']
['2', '0.920313', '0.50293', '0.159375', '0.130859', '0.716543']
['2', '0.367969', '0.504883', '0.304688', '0.130859', '0.804096']
['0', '0.207031', '0.526367', '0.151562', '0.330078', '0.863752']
['0', '0.885156', '0.5

['1', '0.103125', '0.56543', '0.203125', '0.181641', '0.486546']
['1', '0.832031', '0.525391', '0.195312', '0.148438', '0.795217']
['0', '0.830469', '0.474609', '0.0984375', '0.234375', '0.86354']
['1', '0.104687', '0.558594', '0.20625', '0.191406', '0.487896']
['1', '0.832031', '0.525391', '0.195312', '0.148438', '0.795217']
['0', '0.830469', '0.474609', '0.0984375', '0.234375', '0.86354']
['0', '0.361719', '0.46875', '0.110937', '0.246094', '0.485484']
['1', '0.357812', '0.553711', '0.2625', '0.205078', '0.867312']
['0', '0.364062', '0.472656', '0.134375', '0.25', '0.403218']
['1', '0.357031', '0.557617', '0.260938', '0.201172', '0.845834']
['0', '0.360938', '0.470703', '0.121875', '0.25', '0.504977']
['1', '0.357812', '0.555664', '0.2625', '0.205078', '0.864486']
['0', '0.0882813', '0.480469', '0.154687', '0.257812', '0.345216']
['1', '0.121875', '0.557617', '0.24375', '0.208984', '0.54109']
['0', '0.641406', '0.487305', '0.117188', '0.291016', '0.606681']
['1', '0.638281', '0.55566

['0', '0.266406', '0.460938', '0.154687', '0.308594', '0.778461']
['1', '0.809375', '0.599609', '0.275', '0.199219', '0.895622']
['0', '0.80625', '0.517578', '0.125', '0.277344', '0.536417']
['1', '0.290625', '0.574219', '0.2875', '0.1875', '0.62401']
['0', '0.267188', '0.463867', '0.165625', '0.306641', '0.728934']
['1', '0.809375', '0.599609', '0.275', '0.199219', '0.895624']
['1', '0.290625', '0.572266', '0.2875', '0.191406', '0.505259']
['0', '0.80625', '0.517578', '0.125', '0.277344', '0.536417']
['0', '0.265625', '0.460938', '0.15625', '0.3125', '0.77669']
['1', '0.809375', '0.599609', '0.275', '0.199219', '0.895622']
['2', '0.969531', '0.466797', '0.0609375', '0.125', '0.476577']
['0', '0.54375', '0.458008', '0.11875', '0.314453', '0.511809']
['1', '0.577344', '0.567383', '0.279687', '0.193359', '0.673641']
['2', '0.969531', '0.466797', '0.0609375', '0.125', '0.475624']
['0', '0.540625', '0.459961', '0.11875', '0.310547', '0.548946']
['1', '0.577344', '0.568359', '0.279687', '0.

['0', '0.6125', '0.510742', '0.125', '0.458984', '0.521969']
['1', '0.596875', '0.667969', '0.428125', '0.285156', '0.690832']
['1', '0.142969', '0.585938', '0.285937', '0.277344', '0.799859']
['2', '0.821094', '0.459961', '0.354688', '0.259766', '0.834667']
['0', '0.571875', '0.470703', '0.234375', '0.386719', '0.326866']
['2', '0.803906', '0.462891', '0.392188', '0.277344', '0.343039']
['1', '0.6125', '0.613281', '0.409375', '0.324219', '0.733829']
['0', '0.5625', '0.547852', '0.209375', '0.232422', '0.268798']
['2', '0.807813', '0.462891', '0.384375', '0.277344', '0.307068']
['1', '0.6125', '0.613281', '0.4125', '0.324219', '0.730604']
['0', '0.571875', '0.470703', '0.234375', '0.386719', '0.326866']
['2', '0.803906', '0.462891', '0.392188', '0.277344', '0.343039']
['1', '0.6125', '0.613281', '0.409375', '0.324219', '0.733829']
['0', '0.747656', '0.514648', '0.0421875', '0.111328', '0.250547']
['1', '0.688281', '0.519531', '0.0296875', '0.0585938', '0.267969']
['0', '0.847656', '0.5

['0', '0.322656', '0.480469', '0.0234375', '0.0820312', '0.491749']
['2', '0.548437', '0.481445', '0.078125', '0.0800781', '0.506371']
['0', '0.4125', '0.520508', '0.078125', '0.220703', '0.520395']
['0', '0.230469', '0.492188', '0.0484375', '0.132812', '0.539375']
['2', '0.503906', '0.479492', '0.0296875', '0.0488281', '0.544579']
['0', '0.14375', '0.508789', '0.053125', '0.181641', '0.579261']
['1', '0.146875', '0.571289', '0.04375', '0.0957031', '0.616718']
['2', '0.189063', '0.477539', '0.053125', '0.0800781', '0.664537']
['2', '0.0414063', '0.505859', '0.0828125', '0.121094', '0.82266']
['1', '0.864062', '0.575195', '0.190625', '0.130859', '0.865747']
['1', '0.403906', '0.583008', '0.189063', '0.150391', '0.894099']
['0', '0.409375', '0.519531', '0.075', '0.21875', '0.254166']
['0', '0.260938', '0.485352', '0.028125', '0.111328', '0.357163']
['1', '0.219531', '0.52832', '0.0359375', '0.0683594', '0.399232']
['0', '0.8625', '0.510742', '0.103125', '0.236328', '0.446247']
['2', '0.5

...

['0', '0.757031', '0.493164', '0.110937', '0.365234', '0.467803']
['0', '0.633594', '0.583984', '0.0921875', '0.105469', '0.257463']
['0', '0.757031', '0.492188', '0.107813', '0.367188', '0.533084']
['1', '0.933594', '0.602539', '0.132812', '0.244141', '0.885077']
['1', '0.935156', '0.603516', '0.129688', '0.242188', '0.877434']
['1', '0.933594', '0.602539', '0.132812', '0.244141', '0.885077']
['0', '0.583594', '0.47168', '0.0171875', '0.0722656', '0.257148']
['0', '0.592969', '0.464844', '0.0265625', '0.0820312', '0.27903']
['7', '0.830469', '0.432617', '0.114063', '0.0917969', '0.312048']
['2', '0.335156', '0.464844', '0.0453125', '0.0429688', '0.321233']
['0', '0.958594', '0.518555', '0.0484375', '0.126953', '0.36546']
['0', '0.610156', '0.46582', '0.0296875', '0.0800781', '0.399591']
['0', '0.635938', '0.458984', '0.028125', '0.0859375', '0.632875']
['0', '0.541406', '0.462891', '0.0203125', '0.0625', '0.763968']
['0', '0.583594', '0.47168', '0.0171875', '0.0722656', '0.257212']
['

['9', '0.755469', '0.238281', '0.0296875', '0.09375', '0.288574']
['0', '0.839844', '0.369141', '0.0203125', '0.0625', '0.412223']
['0', '0.157031', '0.59082', '0.126563', '0.357422', '0.699747']
['2', '0.791406', '0.506836', '0.417188', '0.212891', '0.786933']
['0', '0.376563', '0.663086', '0.1875', '0.439453', '0.820093']
['0', '0.0742188', '0.660156', '0.145313', '0.347656', '0.842479']
['0', '0.532031', '0.584961', '0.173438', '0.384766', '0.878917']
['9', '0.755469', '0.238281', '0.0296875', '0.09375', '0.288574']
['0', '0.839844', '0.369141', '0.0203125', '0.0625', '0.412223']
['0', '0.155469', '0.591797', '0.126563', '0.355469', '0.623609']
['2', '0.791406', '0.506836', '0.417188', '0.212891', '0.786934']
['0', '0.376563', '0.663086', '0.1875', '0.439453', '0.818445']
['0', '0.0742188', '0.657227', '0.145313', '0.353516', '0.835132']
['0', '0.532031', '0.584961', '0.173438', '0.384766', '0.879008']
['9', '0.755469', '0.238281', '0.0296875', '0.09375', '0.288574']
['0', '0.839844

['0', '0.804688', '0.620117', '0.05625', '0.208984', '0.280796']
['0', '0.892969', '0.583008', '0.0921875', '0.255859', '0.291212']
['0', '0.740625', '0.550781', '0.115625', '0.25', '0.303776']
['2', '0.172656', '0.490234', '0.307813', '0.347656', '0.275661']
['2', '0.721094', '0.483398', '0.0296875', '0.0488281', '0.361866']
['2', '0.59375', '0.503906', '0.23125', '0.289062', '0.376093']
['0', '0.145313', '0.503906', '0.184375', '0.429688', '0.403359']
['2', '0.877344', '0.483398', '0.245312', '0.185547', '0.460843']
['3', '0.138281', '0.657227', '0.170312', '0.392578', '0.473888']
['2', '0.33125', '0.477539', '0.121875', '0.169922', '0.500674']
['2', '0.775781', '0.46875', '0.0921875', '0.132812', '0.546187']
['2', '0.417188', '0.473633', '0.146875', '0.150391', '0.761623']
['2', '0.94375', '0.516602', '0.1125', '0.162109', '0.862828']
['2', '0.721094', '0.483398', '0.0296875', '0.0488281', '0.361864']
['2', '0.267188', '0.492188', '0.1625', '0.195312', '0.364992']
['2', '0.59375', '

['2', '0.191406', '0.473633', '0.307813', '0.232422', '0.311516']
['7', '0.603125', '0.506836', '0.24375', '0.314453', '0.322958']
['2', '0.3625', '0.479492', '0.05', '0.150391', '0.325304']
['2', '0.776563', '0.470703', '0.08125', '0.140625', '0.49315']
['2', '0.882031', '0.482422', '0.232812', '0.191406', '0.506211']
['2', '0.335156', '0.478516', '0.0859375', '0.171875', '0.618632']
['2', '0.953125', '0.512695', '0.09375', '0.189453', '0.626416']
['2', '0.409375', '0.470703', '0.15', '0.15625', '0.685014']
['1', '0.227344', '0.618164', '0.167187', '0.279297', '0.725917']
['0', '0.217187', '0.511719', '0.128125', '0.308594', '0.743114']
['2', '0.257812', '0.494141', '0.1625', '0.273438', '0.30092']
['7', '0.108594', '0.481445', '0.198437', '0.248047', '0.373952']
['2', '0.344531', '0.476562', '0.0640625', '0.171875', '0.418493']
['7', '0.603125', '0.505859', '0.25', '0.316406', '0.473713']
['2', '0.957812', '0.511719', '0.084375', '0.199219', '0.495058']
['0', '0.254687', '0.509766', 

['2', '0.359375', '0.494141', '0.05', '0.0664062', '0.252535']
['2', '0.486719', '0.513672', '0.170312', '0.214844', '0.280972']
['0', '0.657031', '0.517578', '0.0453125', '0.121094', '0.316185']
['7', '0.0609375', '0.504883', '0.121875', '0.423828', '0.370205']
['2', '0.324219', '0.494141', '0.0890625', '0.0898438', '0.432859']
['2', '0.633594', '0.492188', '0.0796875', '0.0859375', '0.434637']
['1', '0.46875', '0.620117', '0.05', '0.158203', '0.443023']
['7', '0.879687', '0.509766', '0.240625', '0.382812', '0.446676']
['2', '0.586719', '0.505859', '0.0484375', '0.0664062', '0.448706']
['0', '0.464844', '0.535156', '0.0734375', '0.257812', '0.69058']
['2', '0.655469', '0.519531', '0.0859375', '0.117188', '0.699332']
['2', '0.740625', '0.514648', '0.06875', '0.166016', '0.71594']
['0', '0.164844', '0.525391', '0.0640625', '0.246094', '0.758162']
['2', '0.320312', '0.485352', '0.06875', '0.0800781', '0.251809']
['7', '0.228125', '0.501953', '0.15', '0.15625', '0.305854']
['7', '0.890625

In [39]:
scores = yolo_scores(path)

In [40]:
scores

[['i',
  'm',
  'a',
  'g',
  'e',
  's',
  ['i', 'm', 'a', 'g', 'e', 0.862248],
  ['i', 'm', 'a', 'g', 'e', 0.847822],
  ['i', 'm', 'a', 'g', 'e', 0.863752]],
 ['i',
  'm',
  'a',
  'g',
  'e',
  's',
  ['i', 'm', 'a', 'g', 'e', 0.703421],
  ['i', 'm', 'a', 'g', 'e', 0.591553],
  ['i', 'm', 'a', 'g', 'e']],
 ['i',
  'm',
  'a',
  'g',
  'e',
  's',
  ['i', 'm', 'a', 'g', 'e'],
  ['i', 'm', 'a', 'g', 'e'],
  ['i', 'm', 'a', 'g', 'e']],
 ['i',
  'm',
  'a',
  'g',
  'e',
  's',
  ['i', 'm', 'a', 'g', 'e', 0.810309],
  ['i', 'm', 'a', 'g', 'e', 0.410738],
  ['i', 'm', 'a', 'g', 'e', 0.810309]],
 ['i',
  'm',
  'a',
  'g',
  'e',
  's',
  ['i', 'm', 'a', 'g', 'e', 0.284495, 0.69883],
  ['i', 'm', 'a', 'g', 'e', 0.698382],
  ['i', 'm', 'a', 'g', 'e', 0.836365]],
 ['i',
  'm',
  'a',
  'g',
  'e',
  's',
  ['i', 'm', 'a', 'g', 'e', 0.836365],
  ['i', 'm', 'a', 'g', 'e', 0.310581],
  ['i', 'm', 'a', 'g', 'e', 0.446006]],
 ['i',
  'm',
  'a',
  'g',
  'e',
  's',
  ['i', 'm', 'a', 'g', 'e'],
